# Deep Active Learning on Mnist and Cifar10 Datasets

In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.activations import softmax

In [13]:
# Load MNIST dataset
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()
input_shape_mnist = x_train_mnist.shape[1:] + (1,)

num_classes_mnist = len(np.unique(y_train_mnist))

y_train_mnist = to_categorical(y_train_mnist, num_classes_mnist)
y_test_mnist = to_categorical(y_test_mnist, num_classes_mnist)

In [14]:
# Load CIFAR-10 dataset
(x_train_cifar, y_train_cifar), (x_test_cifar, y_test_cifar) = cifar10.load_data()
input_shape_cifar = x_train_cifar.shape[1:]

num_classes_cifar = len(np.unique(y_train_cifar))

y_train_cifar = to_categorical(y_train_cifar, num_classes_cifar)
y_test_cifar = to_categorical(y_test_cifar, num_classes_cifar)

In [15]:
def create_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [16]:
def uncertainty_sampling(model, X):
    predictions = model.predict(X)
    uncertainties = np.max(predictions, axis=1)
    return np.argsort(uncertainties)[:10]

def margin_sampling(model, X):
    predictions = model.predict(X)
    margins = np.sort(predictions, axis=1)[:,-1] - np.sort(predictions, axis=1)[:,-2]
    return np.argsort(margins)[:10]

def entropy_sampling(model, X):
    predictions = model.predict(X)
    entropies = -np.sum(predictions * np.log2(predictions + 1e-10), axis=1)
    return np.argsort(entropies)[:10]

In [17]:
def active_learning(strategy, x_train, y_train, x_test, y_test, initial_labeled_samples, iterations, input_shape, num_classes):
    x_labeled = x_train[:initial_labeled_samples]
    y_labeled = y_train[:initial_labeled_samples]

    model = create_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_labeled, y_labeled, epochs=30, batch_size=128, validation_data=(x_test, y_test))
    l, acc = model.evaluate(x_test, y_test)
    print("Initial Accuracy :")
    print(acc)
    print("-----------------------------------------------------------------------------------------")

    overall_accuracy = []
    for i in range(iterations):
        print("")
        print(f"Iteration {i+1} :")

        indices = strategies[strategy](model, x_train)

        x_selected = x_train[indices]
        y_selected = y_train[indices]

        x_train = np.delete(x_train, indices, axis=0)
        y_train = np.delete(y_train, indices, axis=0)

        x_labeled = np.concatenate([x_labeled, x_selected], axis=0)
        y_labeled = np.concatenate([y_labeled, y_selected], axis=0)

        model.fit(x_labeled, y_labeled, epochs=30, batch_size=128, validation_data=(x_test, y_test))

        _, acc = model.evaluate(x_test, y_test)
        overall_accuracy.append(acc)
        print("---------------------------------------------------------------------")
        print(f"Iteration {i+1} -> Accuracy: {acc:.2f}")
        print("---------------------------------------------------------------------")

    print("")
    overall_acc = sum(overall_accuracy) / len(overall_accuracy)
    print(f"Overall Accuracy of the Strategy : {overall_acc:.2f}")
    
    return overall_acc

In [18]:
dataset1 = {'MNIST': (x_train_mnist, y_train_mnist, x_test_mnist, y_test_mnist, input_shape_mnist, num_classes_mnist)}    
dataset2 = {'CIFAR-10': (x_train_cifar, y_train_cifar, x_test_cifar, y_test_cifar, input_shape_cifar, num_classes_cifar)}

strategies = {'Uncertainty': uncertainty_sampling, 'Margin': margin_sampling, 'Entropy': entropy_sampling}

In [19]:
# Run DeepAL Strategies on MNIST Dataset
results_mnist = []
for dataset_name, dataset in dataset1.items():
    x_train, y_train, x_test, y_test, input_shape, num_classes = dataset
    for strategy in strategies:
        print("")
        print("------------------------------------------------------------------------------------------------")
        print(f"Running DeepAL Using {strategy} Strategy for MNIST dataset...")
        print("------------------------------------------------------------------------------------------------")
        accuracy = active_learning(strategy, x_train, y_train, x_test, y_test, initial_labeled_samples=1000, 
                                   iterations=50, input_shape=input_shape, num_classes=num_classes)
        results_mnist.append({'Dataset': dataset_name, 'Strategy': strategy, 'Accuracy': accuracy})

results_mnist = pd.DataFrame(results_mnist)

Streaming output truncated to the last 5000 lines.
10/10 [==============================] - 0s 34ms/step - loss: 0.0072 - accuracy: 0.9969 - val_loss: 0.1490 - val_accuracy: 0.9790
Epoch 15/30
10/10 [==============================] - 0s 33ms/step - loss: 0.0111 - accuracy: 0.9969 - val_loss: 0.1514 - val_accuracy: 0.9778
Epoch 16/30
10/10 [==============================] - 0s 34ms/step - loss: 0.0101 - accuracy: 0.9969 - val_loss: 0.1514 - val_accuracy: 0.9775
Epoch 17/30
10/10 [==============================] - 0s 44ms/step - loss: 0.0160 - accuracy: 0.9929 - val_loss: 0.1533 - val_accuracy: 0.9782
Epoch 18/30
10/10 [==============================] - 0s 44ms/step - loss: 0.0103 - accuracy: 0.9969 - val_loss: 0.1569 - val_accuracy: 0.9788
Epoch 19/30
10/10 [==============================] - 0s 45ms/step - loss: 0.0102 - accuracy: 0.9969 - val_loss: 0.1546 - val_accuracy: 0.9797
Epoch 20/30
10/10 [==============================] - 0s 45ms/step - loss: 0.0095 - accuracy: 0.9969 - val_los

In [20]:
# Run DeepAL Strategies on CIFAR10 Dataset
results_cifar = []
for dataset_name, dataset in dataset2.items():
    x_train, y_train, x_test, y_test, input_shape, num_classes = dataset
    for strategy in strategies:
        print("")
        print("------------------------------------------------------------------------------------------------")
        print(f"Running DeepAL Using {strategy} Strategy for CIFAR10 dataset...")
        print("------------------------------------------------------------------------------------------------")
        accuracy = active_learning(strategy, x_train, y_train, x_test, y_test, initial_labeled_samples=1000, 
                                   iterations=50, input_shape=input_shape, num_classes=num_classes)
        results_cifar.append({'Dataset': dataset_name, 'Strategy': strategy, 'Accuracy': accuracy})

results_cifar = pd.DataFrame(results_cifar)

Streaming output truncated to the last 5000 lines.
10/10 [==============================] - 0s 46ms/step - loss: 0.0772 - accuracy: 0.9748 - val_loss: 6.2065 - val_accuracy: 0.3944
Epoch 15/30
10/10 [==============================] - 0s 46ms/step - loss: 0.0916 - accuracy: 0.9717 - val_loss: 6.6083 - val_accuracy: 0.3834
Epoch 16/30
10/10 [==============================] - 0s 38ms/step - loss: 0.0759 - accuracy: 0.9803 - val_loss: 6.0903 - val_accuracy: 0.3927
Epoch 17/30
10/10 [==============================] - 0s 48ms/step - loss: 0.1022 - accuracy: 0.9732 - val_loss: 5.6975 - val_accuracy: 0.3907
Epoch 18/30
10/10 [==============================] - 0s 39ms/step - loss: 0.0712 - accuracy: 0.9772 - val_loss: 5.7489 - val_accuracy: 0.3936
Epoch 19/30
10/10 [==============================] - 0s 39ms/step - loss: 0.0874 - accuracy: 0.9787 - val_loss: 5.9944 - val_accuracy: 0.3910
Epoch 20/30
10/10 [==============================] - 0s 37ms/step - loss: 0.0957 - accuracy: 0.9717 - val_los

In [23]:
print("Results for MNIST dataset:")
print("------------------------------")
print(results_mnist)

Results for MNIST dataset:
------------------------------
  Dataset     Strategy  Accuracy
0   MNIST  Uncertainty  0.973294
1   MNIST       Margin  0.974860
2   MNIST      Entropy  0.959206


In [22]:
print("Results for CIFAR10 dataset:")
print("------------------------------")
print(results_cifar)

Results for CIFAR10 dataset:
------------------------------
    Dataset     Strategy  Accuracy
0  CIFAR-10  Uncertainty  0.395614
1  CIFAR-10       Margin  0.393670
2  CIFAR-10      Entropy  0.387508


# Deep Active Learning on an Unbalanced Dataset

In [25]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=10000, n_features=20, n_informative=10, n_redundant=0, n_classes=2, weights=[0.7, 0.3])
x_train_unbalanced, x_test_unbalanced, y_train_unbalanced, y_test_unbalanced = train_test_split(X, y, test_size=0.2, stratify=y)

input_shape_unbalanced = x_train_unbalanced.shape[1:]

num_classes_unbalanced = len(np.unique(y_train_unbalanced))

y_train_unbalanced = to_categorical(y_train_unbalanced, num_classes_unbalanced)
y_test_unbalanced = to_categorical(y_test_unbalanced, num_classes_unbalanced)

In [26]:
def create_model(input_shape, num_classes):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

def uncertainty_sampling(model, X):
    predictions = model.predict(X)
    uncertainties = np.max(predictions, axis=1)
    return np.argsort(uncertainties)[:10]

def margin_sampling(model, X):
    predictions = model.predict(X)
    margins = np.sort(predictions, axis=1)[:,-1] - np.sort(predictions, axis=1)[:,-2]
    return np.argsort(margins)[:10]

def entropy_sampling(model, X):
    predictions = model.predict(X)
    entropies = -np.sum(predictions * np.log2(predictions + 1e-10), axis=1)
    return np.argsort(entropies)[:10]

In [27]:
def active_learning(strategy, x_train, y_train, x_test, y_test, initial_labeled_samples, iterations, input_shape, num_classes):
    labeled_samples_per_class = initial_labeled_samples // num_classes
    x_labeled = []
    y_labeled = []
    for i in range(num_classes):
        indices = np.where(y_train[:, i] == 1)[0][:labeled_samples_per_class]
        x_labeled.append(x_train[indices])
        y_labeled.append(y_train[indices])
    x_labeled = np.concatenate(x_labeled, axis=0)
    y_labeled = np.concatenate(y_labeled, axis=0)

    model = create_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_labeled, y_labeled, epochs=20, batch_size=128, validation_data=(x_test, y_test))
    _, acc = model.evaluate(x_test, y_test)
    print("Initial Accuracy :")
    print(acc)
    print("-----------------------------------------------------------------------------------------")

    overall_accuracy = []
    for i in range(iterations):
        print("")
        print(f"Iteration {i+1} :")

        indices = strategies[strategy](model, x_train)

        x_selected = x_train[indices]
        y_selected = y_train[indices]

        x_train = np.delete(x_train, indices, axis=0)
        y_train = np.delete(y_train, indices, axis=0)

        for j in range(num_classes):
            indices = np.where(y_selected[:, j] == 1)[0]
            x_labeled_j = x_selected[indices]
            y_labeled_j = y_selected[indices]
            x_labeled = np.concatenate([x_labeled, x_labeled_j], axis=0)
            y_labeled = np.concatenate([y_labeled, y_labeled_j], axis=0)

        model.fit(x_labeled, y_labeled, epochs=20, batch_size=128, validation_data=(x_test, y_test))

        _, acc = model.evaluate(x_test, y_test)
        overall_accuracy.append(acc)
        print("---------------------------------------------------------------------")
        print(f"Iteration {i+1} -> Accuracy: {acc:.2f}")
        print("---------------------------------------------------------------------")

    print("")
    overall_acc = sum(overall_accuracy) / len(overall_accuracy)
    print(f"Overall Accuracy of the Strategy : {overall_acc:.2f}")
    
    return overall_acc

In [28]:
strategies = {'Uncertainty': uncertainty_sampling, 'Margin': margin_sampling, 'Entropy': entropy_sampling}

In [29]:
# Run DeepAL Strategies on unbalanced dataset
results_unbalanced = []
for strategy in strategies:
    print("")
    print("------------------------------------------------------------------------------------------------")
    print(f"Running DeepAL Using {strategy} Strategy for unbalanced dataset...")
    print("------------------------------------------------------------------------------------------------")
    accuracy = active_learning(strategy, x_train_unbalanced, y_train_unbalanced, x_test_unbalanced, y_test_unbalanced, 
                               initial_labeled_samples=1000, iterations=30, input_shape=input_shape_unbalanced, 
                               num_classes=num_classes_unbalanced)
    results_unbalanced.append({'Strategy': strategy, 'Accuracy': accuracy})

results_unbalanced = pd.DataFrame(results_unbalanced)


------------------------------------------------------------------------------------------------
Running DeepAL Using Uncertainty Strategy for unbalanced dataset...
------------------------------------------------------------------------------------------------
Epoch 1/20
8/8 [==============================] - 2s 34ms/step - loss: 0.7979 - accuracy: 0.5690 - val_loss: 0.4978 - val_accuracy: 0.7925
Epoch 2/20
8/8 [==============================] - 0s 10ms/step - loss: 0.6586 - accuracy: 0.6500 - val_loss: 0.4243 - val_accuracy: 0.8305
Epoch 3/20
8/8 [==============================] - 0s 10ms/step - loss: 0.5781 - accuracy: 0.7220 - val_loss: 0.3988 - val_accuracy: 0.8370
Epoch 4/20
8/8 [==============================] - 0s 15ms/step - loss: 0.5351 - accuracy: 0.7510 - val_loss: 0.3691 - val_accuracy: 0.8470
Epoch 5/20
8/8 [==============================] - 0s 10ms/step - loss: 0.5247 - accuracy: 0.7540 - val_loss: 0.3394 - val_accuracy: 0.8590
Epoch 6/20
8/8 [==========================

In [30]:
print("Results for UNBALANCED Dataset:")
print("------------------------------")
print(results_unbalanced)

Results for UNBALANCED Dataset:
------------------------------
      Strategy  Accuracy
0  Uncertainty  0.947500
1       Margin  0.948750
2      Entropy  0.930533


# Deep Active Learning with a deep NLP model

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.activations import softmax

In [2]:
# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
maxlen = 100
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

num_classes = 2
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [3]:
def create_model(input_shape, num_classes):
    model = Sequential([
        Embedding(10000, 16, input_length=maxlen),
        LSTM(32, dropout=0.5, recurrent_dropout=0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [4]:
def uncertainty_sampling(model, X):
    predictions = model.predict(X)
    uncertainties = np.max(predictions, axis=1)
    return np.argsort(uncertainties)[:10]

def margin_sampling(model, X):
    predictions = model.predict(X)
    margins = np.sort(predictions, axis=1)[:,-1] - np.sort(predictions, axis=1)[:,-2]
    return np.argsort(margins)[:10]

def entropy_sampling(model, X):
    predictions = model.predict(X)
    entropies = -np.sum(predictions * np.log2(predictions + 1e-10), axis=1)
    return np.argsort(entropies)[:10]

In [5]:
def active_learning(strategy, x_train, y_train, x_test, y_test, initial_labeled_samples, iterations, input_shape, num_classes):
    x_labeled = x_train[:initial_labeled_samples]
    y_labeled = y_train[:initial_labeled_samples]

    model = create_model(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_labeled, y_labeled, epochs=10, batch_size=128, validation_data=(x_test, y_test))
    l, acc = model.evaluate(x_test, y_test)
    print("Initial Accuracy :")
    print(acc)
    print("-----------------------------------------------------------------------------------------")

    overall_accuracy = []
    for i in range(iterations):
        print("")
        print(f"Iteration {i+1} :")

        indices = strategies[strategy](model, x_train)

        x_selected = x_train[indices]
        y_selected = y_train[indices]

        x_train = np.delete(x_train, indices, axis=0)
        y_train = np.delete(y_train, indices, axis=0)

        x_labeled = np.concatenate([x_labeled, x_selected], axis=0)
        y_labeled = np.concatenate([y_labeled, y_selected], axis=0)

        model.fit(x_labeled, y_labeled, epochs=10, batch_size=128, validation_data=(x_test, y_test))

        _, acc = model.evaluate(x_test, y_test)
        overall_accuracy.append(acc)
        print("---------------------------------------------------------------------")
        print(f"Iteration {i+1} -> Accuracy: {acc:.2f}")
        print("---------------------------------------------------------------------")

    print("")
    overall_acc = sum(overall_accuracy) / len(overall_accuracy)
    print(f"Overall Accuracy of the Strategy : {overall_acc:.2f}")
    
    return overall_acc

strategies = {'Uncertainty': uncertainty_sampling, 'Margin': margin_sampling, 'Entropy': entropy_sampling}

In [6]:
# Run DeepAL Strategies on IMDB Dataset
results_imdb = []
for strategy in strategies:
    print("")
    print("------------------------------------------------------------------------------------------------")
    print(f"Running DeepAL Using {strategy} Strategy for IMDB dataset...")
    print("------------------------------------------------------------------------------------------------")
    accuracy = active_learning(strategy, x_train, y_train, x_test, y_test, initial_labeled_samples=1000, 
                               iterations=30, input_shape=(maxlen,), num_classes=num_classes)
    results_imdb.append({'Dataset': 'IMDB', 'Strategy': strategy, 'Accuracy': accuracy})

results_imdb = pd.DataFrame(results_imdb)


------------------------------------------------------------------------------------------------
Running DeepAL Using Uncertainty Strategy for IMDB dataset...
------------------------------------------------------------------------------------------------
Epoch 1/10
8/8 [==============================] - 5s 365ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
8/8 [==============================] - 2s 326ms/step - loss: 0.6924 - accuracy: 0.5140 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 3/10
8/8 [==============================] - 2s 324ms/step - loss: 0.6910 - accuracy: 0.5340 - val_loss: 0.6921 - val_accuracy: 0.5234
Epoch 4/10
8/8 [==============================] - 3s 486ms/step - loss: 0.6882 - accuracy: 0.5840 - val_loss: 0.6908 - val_accuracy: 0.5228
Epoch 5/10
8/8 [==============================] - 4s 494ms/step - loss: 0.6831 - accuracy: 0.5970 - val_loss: 0.6876 - val_accuracy: 0.5735
Epoch 6/10
8/8 [===========================

In [7]:
print("Results for IMDB dataset (NLP Model):")
print("------------------------------")
print(results_imdb)

Results for IMDB dataset (NLP Model):
------------------------------
  Dataset     Strategy  Accuracy
0    IMDB  Uncertainty  0.766167
1    IMDB       Margin  0.762612
2    IMDB      Entropy  0.748751
